# WebsocketServerWorker Tutorial

This tutorial is a 2 notebook tutorial. The partner notebook is the notebook entitled SocketWorker Client.ipynb and is in the same folder as this notebook. You should execute this notebook BEFORE the other.

In this tutorial, we'll demonstrate how to launch a WebsocketWorker server which will listen for PyTorch commands over a socket connection. In this tutorial, the two workers are connected via a socket connection on the localhost network.

If you'd prefer to download this notebook and run it locally on your machine, you can do so via the `Download .ipynb` in the `File` dropdown field in this Google colab environment. 


# Step -1: Copy This Notebook

Go up to File -> Save A Copy in Drive

This will let you execute the notebook (it won't let you execute this one by default)

# Step 0: Install Dependencies

In [ ]:
! git clone https://github.com/OpenMined/PySyft.git
! git checkout dev
! git pull origin dev
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
#!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

import torch

!cd PySyft; pip install -r requirements.txt; python setup.py install

import os
import sys
import logging

In [ ]:
print(sys.path)

Tweak the following to make things work if necessary. 

In [ ]:
#sys.path.remove('/anaconda3/envs/pysyft/lib/python3.7/site-packages/syft-0.1.21a1-py3.7.egg')
#sys.path.append('./PySyft')
#print(sys.path)

# Step 1: Hook PyTorch

Just like previous tutorials, the first step is to override PyTorch commands using the TorchHook object.

In [ ]:
import syft as sy

hook = sy.TorchHook(torch)

# Step 2: Launch Server

The next step is to launch the server. We set the `host` to "localhost", `hook` to the "hook" set above, `id` to the "id" of the worker, and `port` to the worker's "port" (in this case the port is at `8182`). 

In [ ]:
from syft.workers.websocket_server import WebsocketServerWorker

local_worker = WebsocketServerWorker(
                            host="localhost",
                            hook=hook,
                            id=0,
                            port=8182)

local_worker.start()  # Might need to interrupt with `CTRL-C` or some other means

In [ ]:
local_worker.list_objects()

In [ ]:
local_worker.objects_count()

In [ ]:
local_worker.host

In [ ]:
local_worker.port